In [2]:
import pandas as pd
from database import engine

%load_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
def query_data():
    query = """
    select
	    strdrink,
	    stralcoholic,
	    strglass,
	    stringredient1,
	    stringredient2,
	    stringredient3,
	    stringredient4,
	    stringredient5,
	    stringredient6,
	    stringredient7,
	    stringredient8,
	    stringredient9,
	    stringredient10,
	    stringredient11,
	    stringredient12,
	    strmeasure1_clean,
	    strmeasure2_clean,
	    strmeasure3_clean,
	    strmeasure4_clean,
	    strmeasure5_clean,
	    strmeasure6_clean,
	    strmeasure7_clean,
	    strmeasure8_clean,
	    strmeasure9_clean,
	    strmeasure10_clean,
	    strmeasure11_clean,
        strmeasure12_clean
	    
	from 
	    all_cocktails
	;
        """

    return pd.read_sql_query(query, engine)

<IPython.core.display.Javascript object>

In [35]:
df = query_data()

<IPython.core.display.Javascript object>

In [18]:
display(df.isna().sum())
display(df.stralcoholic.value_counts())
df.strglass.value_counts()
df.info()

strdrink                0
stralcoholic            0
strglass                0
stringredient1          0
stringredient2          0
stringredient3         89
stringredient4        272
stringredient5        421
stringredient6        528
stringredient7        592
stringredient8        613
stringredient9        622
stringredient10       625
stringredient11       625
stringredient12       627
strmeasure1_clean       0
strmeasure2_clean       0
strmeasure3_clean       0
strmeasure4_clean       0
strmeasure5_clean       0
strmeasure6_clean       0
strmeasure7_clean       0
strmeasure8_clean       0
strmeasure9_clean       0
strmeasure10_clean      0
strmeasure11_clean      0
strmeasure12_clean      0
dtype: int64

Alcoholic           562
Non alcoholic        57
Optional alcohol      9
Name: stralcoholic, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 628 entries, 0 to 627
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   strdrink            628 non-null    object 
 1   stralcoholic        628 non-null    object 
 2   strglass            628 non-null    object 
 3   stringredient1      628 non-null    object 
 4   stringredient2      628 non-null    object 
 5   stringredient3      539 non-null    object 
 6   stringredient4      356 non-null    object 
 7   stringredient5      207 non-null    object 
 8   stringredient6      100 non-null    object 
 9   stringredient7      36 non-null     object 
 10  stringredient8      15 non-null     object 
 11  stringredient9      6 non-null      object 
 12  stringredient10     3 non-null      object 
 13  stringredient11     3 non-null      object 
 14  stringredient12     1 non-null      object 
 15  strmeasure1_clean   628 non-null    float64
 16  strmeasu

<IPython.core.display.Javascript object>

In [25]:
ingred_cols = [col for col in df.columns if col.startswith("stringredient")]

measure_cols = [col for col in df.columns if col.startswith("strmeasure")]

<IPython.core.display.Javascript object>

In [48]:
# Convert all ingredients to lowercase
ingred_lower = df[ingred_cols].apply(lambda col: col.str.lower())

ingred_clean = (
    df[["strdrink"]]
    .merge(ingred_lower, left_index=True, right_index=True)
    .melt(id_vars="strdrink", value_vars=ingred_cols)
    .drop("variable", axis=1)
    .sort_values("strdrink")
    .dropna(subset=["value"])
    .reset_index(drop=True)
)

ingred_clean

,strdrink,value
0,1-900-FUK-MEUP,midori melon liqueur
1,1-900-FUK-MEUP,absolut kurant
2,1-900-FUK-MEUP,grand marnier
3,1-900-FUK-MEUP,chambord raspberry liqueur
4,1-900-FUK-MEUP,cranberry juice
...,...,...
2517,Zorbatini,vodka
2518,Zorbatini,ouzo
2519,Zorro,sambuca
2520,Zorro,baileys irish cream


<IPython.core.display.Javascript object>

In [64]:
ingredient_dummies = (
    pd.get_dummies(ingred_clean, columns=["value"])
    .groupby("strdrink")
    .sum()
    .reset_index()
)

<IPython.core.display.Javascript object>

In [65]:
ingredient_dummies

,strdrink,value_151 proof rum,value_7-up,value_absinthe,value_absolut citron,value_absolut kurant,value_absolut peppar,value_absolut vodka,value_advocaat,value_agave syrup,...,value_white rum,value_white wine,value_wild turkey,value_wine,value_worcestershire sauce,value_wormwood,value_yellow chartreuse,value_yoghurt,value_yukon jack,value_zima
0,1-900-FUK-MEUP,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,110 in the shade,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,151 Florida Bushwacker,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,155 Belmont,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24k nightmare,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,Zizi Coin-coin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
624,Zoksel,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
625,Zombie,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
626,Zorbatini,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>

Left off here - the final two steps to make this work (before checking it that it is actually right) are as follows:
(1) Need to multiply these by the proportions Kashaf produced.
(2) Need to calculate row sums and get proportions for each drink

After that, we have the data matrix and need to figure out a good way to visualize or analyze this (Ed mentioned doing some Cos analysis)